# Gallery Example: M/MAP/1 Queue

This example demonstrates an M/MAP/1 queueing system:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Markovian Arrival Process (MAP)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

In this queue, the service times follow a MAP distribution while arrivals are Poisson.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mmap1(map_dist=None): Network {
    """Create M/MAP/1 queueing model"""
    if map_dist is None:
        val map_dist = MAP.rand()
        map_dist.set_mean(0.5)  # Scale MAP to have mean service time of 0.5
    
    val model = Network("M/MAP/1")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
    source.setArrival(oclass, Exp(1))  # Poisson arrivals with rate 1
    queue.setService(oclass, map_dist)  # MAP service
// Block 3: topology
    model.link(Network.serialRouting(source, queue, sink))
    
    return model
// Create the model
val model = gallery_mmap1()

## About M/MAP/1 Queues

In the M/MAP/1 queue:
- Arrivals follow a Poisson process (exponential inter-arrivals)
- Service times follow a MAP distribution

The MAP service process can model:
- Service time variability beyond what exponential can capture
- Correlation between consecutive service times
- Phased service operations with state-dependent behavior

With utilization ρ = λ × E[S] where λ=1 and E[S]=0.5, we have ρ=0.5.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)